In [1]:
# establish fixed values
v <- 30
sigma_sq <- 15 
mu <- 35

In [2]:
# inv chi-squared
dist.inv_chi_squared <- function(v, sigma_sq, x) {
    return(((v/2)*sigma_sq)^(v/2) * exp(-(v*sigma_sq)/(2*x))/(gamma(v/2)*x^{1+v/2}))
}

dist.normal <- function(sigma_sq, mu, x) {
    return (1/((2*pi*sigma_sq)^0.5)*exp(-(x-mu)^2/(sigma_sq)))
}

In [122]:
# install.packages("extraDistr") to attain rinvchisq
library(extraDistr)

set.seed(117)
N <- 50

# generating random values from the inverse chi-squared 
# rinvchisq(n, nu, tau)
# V is a vector of V_i
V <- rinvchisq(N, v, sigma_sq)
Y <- rnorm(N, mu, sqrt(V))

In [123]:
Y

[1] 32.32503 35.21666 36.93209 35.66769 39.74576 35.12665 38.83647 36.69711
 [9] 35.30559 29.80719 28.96317 35.15094 37.84780 38.36376 31.48112 30.38745
[17] 36.01614 36.17966 37.09462 33.88485 34.15880 29.35855 38.61571 28.90367
[25] 38.71873 33.21784 35.78779 35.72017 31.52207 39.98281 34.98952 39.76658
[33] 33.15415 38.74888 36.28212 32.64652 32.89798 39.05410 33.20062 34.65758
[41] 30.02248 34.68899 35.39537 31.74837 31.80603 36.57061 33.44723 33.89658
[49] 41.29701 32.27923

May have to perform MH step within-Gibbs to sample from mu, as does not follow any standard distribution

In [116]:
if (FALSE) {
    mhsampler <- function(data, num_params=3, it=10000) {
    n = length(data)
    mchain <- matrix(NA, num_params, it)
    

    
    acc <- 0 # number of accepted proposals
    
    # starting values for Markov-Chain
    mchain[, 1] = c(10, 15, 20)
    v 
    for (i in 2:it) {
        curr_mu <- mchain[1, i-1]
        curr_sigma_sq <- mchain[2, i-1]
        curr_Vi <- mchain[3, i-1]
    
        # sample from full conditional of mu
        curr_mu <- rnorm(n = 1, mean =  curr_mu, sd = curr_Vi^0.5)
        # sample from full conditional of sigma_sq
        curr_sigma_sq <- rgamma(n = 1, shape=(v/2)*curr_Vi, scale=v/2 + 1)
        # sample from full conditional of Vi
        
        curr_Vi <- 1/rgamma(n=1, shape=(n+v)/2, scale=2/(sum(data-curr_mu)^2) + 2/(sum(data-curr_mu)^2) + v*curr_sigma_sq)
        }
    
    return(mchain)
}

mhsampler(Y, num_params=3, it=10)
}

In [117]:
# generic functions
# assume proposal y|x is geneq(x[t])

ITER_NUM = 10

target_dist <- function(args) {
    return(0)
}

propose <- function(x) {
    rnorm(1, mean=x, sd=1)
}

# define an array (single value) 
x_vals <- rep(NA, ITER_NUM)
x_cur <- -1 # starting value

for (iter in 1:ITER_NUM) {
    x_star <- propose(x_cur)
    
    # MH ratio
    ratio <- min(1, target_dist(x_star)/target_dist(x_cur))
    
    # boolean for accept/reject
    
    accept <- runif(1) < ratio
    
    # accept new value if ratio greater than runif(1), else keep current
    x_vals[iter] <- ifelse(accept, x_star, x_cur)
    
    x_cur <- x_vals[iter]
}

Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”Warning message in rnorm(1, mean = x, sd = 1):
“NAs produced”

In [118]:
sample_indep_mh <- function(rg, dg, dq, cur) {
    proposed <- rg() 
    u <- runif(1)
    ratio <- dq(proposed)*dg(cur)/dq(cur)*dg(proposed)
    
    accept = u < ratio

    
    if(accept) {
        return(proposed)
    }
    return(cur)
}

# defining full conditionals  
sample_V <- function(y, mu, v, sigma_sq, N) {
    
    return(rinvchisq(N, 
              v+1, 
              ((y-mu)^2 + sigma_sq*v)/(v+1)))    
}

sample_sigma_sq <- function(v, n, V_i) {
    return(rgamma(1,
                 shape=0.5*n*v + 1,
                  rate=(v/2)*sum(1/V_i)
                 ))
}


sample_mu <- function(Y, V_i, sigma_sq, v, mu) {
    
    mu_mchain <- rep(NA, 1000)
    mu_mchain[1] <- mu
    
    rg <- function() {
        rnorm(1, mu, sigma_sq^0.5)
    }
    
    dg <- function(x) {
        exp(-0.5 * sum((Y-mu)^2/mean(V_i)))
    }
    
    dq <- function(x) {
        return(dnorm(x, mu, sigma_sq^0.5))
    }
    
    for (i in 2:10000) {
            mu_mchain[i] <- sample_indep_mh(rg, dg, dq, mu_mchain[i-1]) 
    } 
    
    return(mean(mu_mchain))
}

ITER_NUM <- 100
data <- Y

mchain <- matrix(NA,ITER_NUM, 2)
# set starting values
# column1=mean, column2=sigma_sq
mchain[1, 1] <- 25
mchain[1, 2] <- 10


V_i <- matrix(NA, ITER_NUM, N)
V_i[1,] <- rep(1, N) 

v <- 30

for(i in 2:ITER_NUM) {
    cur_V_i <- V_i[i-1,]
    
    cur_mu <- mchain[i-1, 1]
    cur_sigma_sq <- mchain[i-1, 2]
    

    new_V_i <- sample_V(data, cur_mu, v, cur_sigma_sq, N)
    
    # MH in Gibbs step    
    new_mu <- sample_mu(data, cur_V_i, cur_sigma_sq, v, cur_mu)
    
    new_sigma_sq <- sample_sigma_sq(v, N, cur_V_i)  

    V_i[i, ] <- new_V_i
    mchain[i, 1] <- new_mu
    mchain[i, 2] <- new_sigma_sq 
  
    
}

In [119]:
mean(mchain[,1])

[1] 25

In [120]:
mean(mchain[,2])

[1] 50.54838

In [121]:
colMeans(V_i) - V

[1] 54.65640 53.46147 54.43895 55.47468 54.10480 56.81701 56.34923 55.35533
 [9] 56.92593 51.18771 53.64559 51.80309 56.50102 53.46645 54.89205 52.84332
[17] 53.39038 56.26668 54.09373 53.16211 55.63966 57.05402 53.70375 52.56542
[25] 52.86943 58.30387 53.79716 55.83077 52.49811 53.62117 51.79742 55.72800
[33] 53.51423 57.86402 54.76459 54.26275 54.32854 53.45485 55.63207 54.12528
[41] 52.64304 55.55180 56.85608 56.84459 57.16378 53.21050 51.86793 55.77411
[49] 55.12031 54.89816